# Interpret LightGBM Classifier with SHAP: Spaceship Titanic Dataset

## Table of Contents

1. [Data Preparation](#data-preparation)
   - [Basic Exploratory Data Analysis (EDA)](#basic-exploratory-data-analysis) 
   - [Import Data and Modules](#import-data-and-modules)
   - [Data Cleaning](#data-cleaning)
   - [Feature Engineering](#feature-engineering)
   - [Advanced Exploratory Data Analysis (EDA)](#advanced-exploratory-data-analysis)
2. [Modeling](#modeling)
   - [Data Preprocessing for Modeling](#data-preprocessing-for-modeling)
   - [Model Training](#model-training)
   - [Model Evaluation](#model-evaluation)
3. [Interpretability](#interpretability)
   - [SHAP Analysis](#shap-analysis)

# 1) Data Preparation
<a id="data-preparation"></a>

## Import Data and Modules
<a id="import-data-and-modules"></a>

In [1]:
# base packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# modeling and evaluation
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import shap
import os

In [2]:
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
df = pd.concat([train, test], ignore_index=True)

## Basic Exploratory Data Analysis (EDA)
<a id="basic-exploratory-data-analysis"></a>

#### 1. Data Description

> * **PassengerId** - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.<br>
> * **HomePlanet** - The planet the passenger departed from, typically their planet of permanent residence.<br>
> * **CryoSleep** - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.<br>
> * **Cabin** - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.<br>
> * **Destination** - The planet the passenger will be debarking to.<br>
> * **Age** - The age of the passenger.<br>
> * **VIP** - Whether the passenger has paid for special VIP service during the voyage.<br>
> * **RoomService**, **FoodCourt**, **ShoppingMall**, **Spa**, **VRDeck** - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.<br>
> * **Name** - The first and last names of the passenger.<br>
> * **Transported** - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.<br>

#### 2. Dataframe Structure

In [3]:
df.shape

(17386, 14)

#### 3. Count Transported Passengers In Training Dataset

In [4]:
transported_counts = train['Transported'].value_counts()
total_passengers = len(train)
transported_percentage = ((transported_counts / total_passengers) * 100).round(2)

transported_df = pd.DataFrame({
    'Count': transported_counts,
    'Percentage': transported_percentage
})

print(transported_df)

             Count  Percentage
Transported                   
True          4378       50.36
False         4315       49.64


#### 4. View Merged Dataframe

In [5]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


#### 5. View Summary Statistics of Continuous Features

In [6]:
summary_stats = df.describe().apply(lambda x: x.round().astype(int))
print(summary_stats)

         Age  RoomService  FoodCourt  ShoppingMall    Spa  VRDeck
count  17028        17024      17020         16970  17020   17010
mean      29          225        458           174    311     305
std       14          667       1611           605   1137    1146
min        0            0          0             0      0       0
25%       19            0          0             0      0       0
50%       27            0          0             0      0       0
75%       38           47         76            27     59      46
max       79        14327      29813         23492  22408   24133


#### 6. View Distribution of Categorical Features

In [7]:
categorical_features = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Cabin']

counts_dict = {}

for feature in categorical_features:
    if feature in df.columns:
        counts_dict[feature] = df[feature].value_counts()
    else:
        counts_dict[feature] = 'Feature not found in the dataset.'

for feature, counts in counts_dict.items():
    print(f'Counts for {feature}:')
    if isinstance(counts, pd.Series):
        display(counts.to_frame().transpose())
    else:
        print(counts)
    print('\n')

Counts for HomePlanet:


HomePlanet,Earth,Europa,Mars
count,9204,4262,3518




Counts for CryoSleep:


CryoSleep,False,True
count,10878,6074




Counts for Destination:


Destination,TRAPPIST-1e,55 Cancri e,PSO J318.5-22
count,11830,3600,1592




Counts for VIP:


VIP,False,True
count,16582,398




Counts for Cabin:


Cabin,G/734/S,G/109/P,B/201/P,G/1368/P,G/981/S,G/1476/S,E/13/S,F/1194/P,B/82/S,D/176/S,...,G/559/P,G/558/P,F/701/P,G/547/S,G/557/P,G/556/P,E/231/S,G/545/S,G/543/S,F/947/P
count,16,14,14,14,14,14,14,14,14,14,...,2,2,2,2,2,2,2,2,2,2


#### 7. Check for Missing Values

In [8]:
# Calculate missing values and their percentage
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage}).round(1)
missing_df = missing_df[missing_df['Missing Values'] > 0].sort_values(by='Missing Values', ascending=False)

print(missing_df)

              Missing Values  Percentage
CryoSleep                434         2.5
ShoppingMall             416         2.4
VIP                      406         2.3
HomePlanet               402         2.3
Name                     400         2.3
Cabin                    398         2.3
VRDeck                   376         2.2
FoodCourt                366         2.1
Spa                      366         2.1
Destination              364         2.1
RoomService              362         2.1
Age                      358         2.1


## Data Cleaning
<a id="data-cleaning"></a>

## Feature Engineering
<a id="feature-engineering"></a>

## Advanced Exploratory Data Analysis (EDA)
<a id="advanced-exploratory-data-analysis"></a>

# 2) Modeling
<a id="modeling"></a>

### Data Preprocessing for Modeling
<a id="data-preprocessing-for-modeling"></a>

### Model Training
<a id="model-training"></a>

### Model Evaluation
<a id="model-evaluation"></a>

# 3) Interpretability
<a id="interpretability"></a>

### SHAP Analysis
<a id="shap-analysis"></a>